<a href="https://colab.research.google.com/github/luckson1/ML/blob/main/trainsdxlipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install autotrain-advanced

In [4]:
!autotrain setup

> INFO    Installing latest transformers@main
> INFO    Successfully installed latest transformers
> INFO    Installing latest peft@main
> INFO    Successfully installed latest peft
> INFO    Installing latest diffusers@main
> INFO    Successfully installed latest diffusers
> INFO    Installing latest trl@main
> INFO    Successfully installed latest trl
> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!autotrain dreambooth \
--model stabilityai/stable-diffusion-xl-base-1.0 \
--output output/ \
--image-path /content/drive/MyDrive/controlnet/input \
--prompt "photo of pf perfume bottle" \
--resolution 512 \
--batch-size 1 \
--num-steps 500 \
--fp16 \
--gradient-accumulation 4 \
--lr 1e-4

> INFO    Namespace(version=False, model='stabilityai/stable-diffusion-xl-base-1.0', revision=None, tokenizer=None, image_path='/content/drive/MyDrive/controlnet/input', class_image_path=None, prompt='photo of pf perfume bottle', class_prompt=None, num_class_images=100, class_labels_conditioning=None, prior_preservation=None, prior_loss_weight=1.0, output='output/', seed=42, resolution=512, center_crop=None, train_text_encoder=None, batch_size=1, sample_batch_size=4, epochs=1, num_steps=500, checkpointing_steps=100000, resume_from_checkpoint=None, gradient_accumulation=4, gradient_checkpointing=None, lr=0.0001, scale_lr=None, scheduler='constant', warmup_steps=0, num_cycles=1, lr_power=1.0, dataloader_num_workers=0, use_8bit_adam=None, adam_beta1=0.9, adam_beta2=0.999, adam_weight_decay=0.01, adam_epsilon=1e-08, max_grad_norm=1.0, allow_tf32=None, prior_generation_precision=None, local_rank=-1, xformers=None, pre_compute_text_embeddings=None, tokenizer_max_length=None, text_encoder_use

In [7]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch

In [8]:
model= "stabilityai/stable-diffusion-xl-base-1.0"
pipe=DiffusionPipeline.from_pretrained(model, torch_dtype=torch.float16)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
pipe.to('cuda')
pipe.load_lora_weights('/content/output', weight_name='pytorch_lora_weights.safetensors')

In [ ]:
prompt="image of perfume pf on a sunday beach sunlight, 4k uHD"

In [11]:
refiner=StableDiffusionXLImg2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-xl-refiner-1.0", torch_dtype=torch.float16)
refiner.to('cuda')

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

StableDiffusionXLImg2ImgPipeline {
  "_class_name": "StableDiffusionXLImg2ImgPipeline",
  "_diffusers_version": "0.21.0.dev0",
  "_name_or_path": "stabilityai/stable-diffusion-xl-refiner-1.0",
  "force_zeros_for_empty_prompt": false,
  "requires_aesthetics_score": true,
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    null,
    null
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    null,
    null
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [13]:

for seed in range(10):
  generator=torch.Generator('cuda').manual_seed(seed)

  image=pipe(prompt=prompt, generator=generator, num_inference_steps=20)
  image=image.images[0]
  image=refiner(prompt=prompt, generator=generator, image=image)
  image=image.images[0]
  image.save(f"images_ref/{seed}.png")

  0%|          | 0/20 [00:00<?, ?it/s]

OutOfMemoryError: ignored